In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from models import AttentionResNet92, AttentionResNet56
from tensorflow.keras import optimizers
import time

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

Found 15743 images belonging to 2 classes.
Found 3934 images belonging to 2 classes.


In [3]:
NAME = "dimentia-residual-attention92-CNN-{}".format(int(time.time()))

# build the model
model = AttentionResNet92(n_classes=1) # here n_classes represent neuron in final layer
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
# plot model graph
#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [6]:
def lr_schedule(epoch):

    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 40: # 120
        lr *= 1e-2
    elif epoch > 20: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [9]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/attention/Implementation/logs/{}".format(NAME))
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=7, min_lr=10e-7, epsilon=0.01, verbose=1)

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=lr_schedule(0)), # (lr=lr_schedule(0)),
              metrics=['accuracy'])

Learning rate:  0.0001


In [12]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [13]:
# fits the model on batches with real-time data augmentation
checkpoint = ModelCheckpoint('best_model_92.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [lr_reducer, early_stopper, checkpoint]

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[lr_reducer, lr_scheduler, checkpoint, tensorboard], initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.


/home/trojan/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/trojan/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Learning rate:  0.0001
Epoch 1/50
   1/1967 [..............................] - ETA: 0s - loss: 3.4262 - accuracy: 0.7500WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 640/1967 [========>.....................] - ETA: 9:57 - loss: 1.3001 - accuracy: 0.6514

KeyboardInterrupt: 

In [ ]:
model = load_model('best_model_92.h5')

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)